In [ ]:
!pip install twint

     |████████████████████████████████| 1.1 MB 12.6 MB/s 
     |████████████████████████████████| 263 kB 46.5 MB/s 
     |████████████████████████████████| 379 kB 63.8 MB/s 
     |████████████████████████████████| 291 kB 48.5 MB/s 
     |████████████████████████████████| 160 kB 46.4 MB/s 
     |████████████████████████████████| 192 kB 51.6 MB/s 
     |████████████████████████████████| 271 kB 34.2 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
  Created wheel for twint: filename=twint-2.1.20-py3-none-any.whl size=33930 sha256=2449b15440ce3d4f5080cb4043175741665496437e0816c8de4a4c2588adb650
  Stored in directory: /root/.cache/pip/wheels/44/fc/77/99887a36b5c265a87516158858697d1a0b8f32c4d4dbddbb24
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=e31316e8c7d6e65883b46df72b7c3ce6af7be045c111a4ff730607761b791d99
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Create

In [ ]:
!pip install feedparser

     |████████████████████████████████| 81 kB 5.0 MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=8d49547645d157d9867b89b64431a4b8d10bb71614fc0ed980e887ded8e529f5
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built sgmllib3k


In [ ]:
!pip install dateparser

     |████████████████████████████████| 288 kB 9.9 MB/s 


In [ ]:
import datetime
import time
import pandas as pd
# from News_Scraper import GoogleNews
import twint
# import nest_asyncio
# nest_asyncio.apply()


In [ ]:
import feedparser
from bs4 import BeautifulSoup
import urllib
from dateparser import parse as parse_date
import requests



class GoogleNews:
    def __init__(self, lang = 'en', country = 'US'):
        self.lang = lang.lower()
        self.country = country.upper()
        self.BASE_URL = 'https://news.google.com/rss'

    def __top_news_parser(self, text):
        """Return subarticles from the main and topic feeds"""
        try:
            bs4_html = BeautifulSoup(text, "html.parser")
            # find all li tags
            lis = bs4_html.find_all('li')
            sub_articles = []
            for li in lis:
                try:
                    sub_articles.append({"url": li.a['href'],
                                         "title": li.a.text,
                                         "publisher": li.font.text})
                except:
                    pass
            return sub_articles
        except:
            return text

    def __ceid(self):
        """Compile correct country-lang parameters for Google News RSS URL"""
        return '?ceid={}:{}&hl={}&gl={}'.format(self.country,self.lang,self.lang,self.country)

    def __add_sub_articles(self, entries):
        for i, val in enumerate(entries):
            if 'summary' in entries[i].keys():
                entries[i]['sub_articles'] = self.__top_news_parser(entries[i]['summary'])
            else:
                entries[i]['sub_articles'] = None
        return entries

    def __scaping_bee_request(self, api_key, url):
        response = requests.get(
            url="https://app.scrapingbee.com/api/v1/",
            params={
                "api_key": api_key,
                "url": url,
                "render_js": "false"
            }
        )
        if response.status_code == 200:
            return response
        if response.status_code != 200:
            raise Exception("ScrapingBee status_code: "  + str(response.status_code) + " " + response.text)

    def __parse_feed(self, feed_url, proxies=None, scraping_bee = None):

        if scraping_bee and proxies:
            raise Exception("Pick either ScrapingBee or proxies. Not both!")

        if proxies:
            r = requests.get(feed_url, proxies = proxies)
        else:
            r = requests.get(feed_url)

        if scraping_bee:
            r = self.__scaping_bee_request(url = feed_url, api_key = scraping_bee)
        else:
            r = requests.get(feed_url)


        if 'https://news.google.com/rss/unsupported' in r.url:
            raise Exception('This feed is not available')

        d = feedparser.parse(r.text)

        if not scraping_bee and not proxies and len(d['entries']) == 0:
            d = feedparser.parse(feed_url)

        return dict((k, d[k]) for k in ('feed', 'entries'))

    def __search_helper(self, query):
        return urllib.parse.quote_plus(query)

    def __from_to_helper(self, validate=None):
        try:
            validate = parse_date(validate).strftime('%Y-%m-%d')
            return str(validate)
        except:
            raise Exception('Could not parse your date')



    def top_news(self, proxies=None, scraping_bee = None):
        """Return a list of all articles from the main page of Google News
        given a country and a language"""
        d = self.__parse_feed(self.BASE_URL + self.__ceid(), proxies=proxies, scraping_bee=scraping_bee)
        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

    def topic_headlines(self, topic: str, proxies=None, scraping_bee=None):
        """Return a list of all articles from the topic page of Google News
        given a country and a language"""
        #topic = topic.upper()
        if topic.upper() in ['WORLD', 'NATION', 'BUSINESS', 'TECHNOLOGY', 'ENTERTAINMENT', 'SCIENCE', 'SPORTS', 'HEALTH']:
            d = self.__parse_feed(self.BASE_URL + '/headlines/section/topic/{}'.format(topic.upper()) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        else:
            d = self.__parse_feed(self.BASE_URL + '/topics/{}'.format(topic) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        if len(d['entries']) > 0:
            return d
        else:
            raise Exception('unsupported topic')

    def geo_headlines(self, geo: str, proxies=None, scraping_bee=None):
        """Return a list of all articles about a specific geolocation
        given a country and a language"""
        d = self.__parse_feed(self.BASE_URL + '/headlines/section/geo/{}'.format(geo) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

    def search(self, query: str, helper = True, when = None, from_ = None, to_ = None, proxies=None, scraping_bee=None):
        """
        Return a list of all articles given a full-text search parameter,
        a country and a language
        :param bool helper: When True helps with URL quoting
        :param str when: Sets a time range for the artiles that can be found
        """

        if when:
            query += ' when:' + when

        if from_ and not when:
            # from_ = self.__from_to_helper(validate=from_)
            query += ' after:' + from_

        if to_ and not when:
            # to_ = self.__from_to_helper(validate=to_)
            query += ' before:' + to_

        if helper == True:
            query = self.__search_helper(query)

        search_ceid = self.__ceid()
        search_ceid = search_ceid.replace('?', '&')

        d = self.__parse_feed(self.BASE_URL + '/search?q={}'.format(query) + search_ceid, proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

In [ ]:
table=pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')
df_list = table[1]
print(df_list)
symbols = df_list['Symbol']

                       Company Name      Symbol                      Sector
0                       Adani Ports  ADANIPORTS              Infrastructure
1                      Asian Paints  ASIANPAINT              Consumer Goods
2                         Axis Bank    AXISBANK                     Banking
3                        Bajaj Auto  BAJAJ-AUTO                  Automobile
4                     Bajaj Finance  BAJFINANCE          Financial Services
5                     Bajaj Finserv  BAJAJFINSV          Financial Services
6                     Bharti Airtel  BHARTIARTL           Telecommunication
7                  Bharat Petroleum        BPCL          Energy - Oil & Gas
8              Britannia Industries   BRITANNIA              Consumer Goods
9                             Cipla       CIPLA             Pharmaceuticals
10                       Coal India   COALINDIA                      Metals
11              Divi's Laboratories    DIVISLAB             Pharmaceuticals
12         D

In [ ]:
list_stocks = ['RELIANCE', 'TCS', 'HDFC', 'HINDUNILVR', 'INFY']
df = pd.DataFrame()

for tic in list_stocks:
  val = df_list.loc[df_list.Symbol == tic]
  df = df.append(val,ignore_index = True)
df.head()

,Company Name,Symbol,Sector
0,Reliance Industries,RELIANCE,Energy - Oil & Gas
1,Tata Consultancy Services,TCS,Information Technology
2,HDFC,HDFC,Financial Services
3,Hindustan Unilever,HINDUNILVR,Consumer Goods
4,Infosys,INFY,Information Technology


In [ ]:
df_list = df
symbols = df_list['Symbol']

In [ ]:
for i in range(len(symbols)):
    ticker = symbols[i]
    name = df_list['Company Name'][i]
    print(ticker, name)
    query = 'intitle:' + str(ticker)
    query2 = 'intitle:' + str(name)
    title = []
    date = []
    df = pd.DataFrame()

    start_date = datetime.datetime.strptime('2011-01-01','%Y-%m-%d')
    end_date = datetime.datetime.strptime('2021-11-01','%Y-%m-%d')
    time_period = end_date - start_date
    days = time_period.days
    freq = 10

    while days >= freq:
        time_period = end_date - start_date
        days = time_period.days
        from_date = start_date
        to_date = start_date + datetime.timedelta(freq)
        if (end_date - to_date).days < 0:
            to_date = end_date

        from_date = from_date.strftime('%Y-%m-%d')
        to_date = to_date.strftime('%Y-%m-%d')
        start_date = start_date + datetime.timedelta(freq)

        gn = GoogleNews()
        s = gn.search(query, helper = True, when = None, from_ = from_date, to_ = to_date, proxies=None, scraping_bee=None)
        for i in range(len(s['entries'])):
            title.append(s['entries'][i]['title'])
            date1 = s['entries'][i]['published_parsed']
            date.append(time.strftime('%Y-%m-%d', date1))

        gn = GoogleNews()
        s = gn.search(query2, helper = True, when = None, from_ = from_date, to_ = to_date, proxies=None, scraping_bee=None)
        for i in range(len(s['entries'])):
            title.append(s['entries'][i]['title'])
            date1 = s['entries'][i]['published_parsed']
            date.append(time.strftime('%Y-%m-%d', date1))

    df['Date'] = date
    df['Title'] = title
    file_name= str(ticker) + " title.csv"
    df.to_csv(file_name)
    print(query,len(df))
    time.sleep(30)

RELIANCE Reliance Industries


In [ ]:
import snscrape.modules.twitter as sntwitter
#Get user input
for ticker in df_list['Symbol']:
    query = ticker
    noOfTweet = 200000
    tweets_list = []
    start_date = "2011-01-01"
    end_date = "2021-11-01"
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query + ' lang:en since:' +  start_date + ' until:' + end_date + ' -filter:links -filter:replies').get_items()):
        if i%1000 == 0:
            print(i)
        if i > int(noOfTweet):
            break
        tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.username])

    #Creating a dataframe from the tweets list above 
    df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
    file_name=str(ticker) + ".csv"
    df = df[['Datetime', 'Text']]
    df.to_csv(file_name)
    print(query, len(df))

ModuleNotFoundError: ignored

In [ ]:
DATE_INTERVAL_LIST=['2011-01-01 2011-07-01',
               '2011-07-01 2012-01-01',
               '2012-01-01 2012-07-01',
               '2012-07-01 2013-01-01',
               '2013-01-01 2013-07-01',
               '2013-07-01 2014-01-01',
               '2014-01-01 2014-07-01',
               '2014-07-01 2015-01-01',
               '2015-01-01 2015-07-01',
               '2015-07-01 2016-01-01',
               '2016-01-01 2016-07-01',
               '2016-07-01 2017-01-01',
               '2017-01-01 2017-07-01',
               '2017-07-01 2018-01-01',
               '2018-01-01 2018-07-01',
               '2018-07-01 2019-01-01',
               '2019-01-01 2019-07-01',
               '2019-07-01 2020-01-01',
               '2020-01-01 2020-07-01',
               '2020-07-01 2021-01-01',
               '2021-01-01 2021-07-01']

In [ ]:
KEYWORDS=df_list['Symbol']
KEYWORDS_CODES = []
for query in KEYWORDS:
    # print(query)
    KEYWORDS_CODES.append(pytrend.suggestions(keyword=query)[0])
df_CODES= pd.DataFrame(KEYWORDS_CODES)
df_CODES

In [ ]:
df_google_trends = pd.DataFrame()
df_trends = pd.DataFrame()
for DATE_INTERVAL in DATE_INTERVAL_LIST:
  print(DATE_INTERVAL)
  KEYWORDS_FULL=df_list['Trends key'].to_list()
  COUNTRY=["US"] #Use this link for iso country code
  CATEGORY=0 # Use this link to select categories
  SEARCH_TYPE='' #default is 'web searches',others include 'images','news','youtube','froogle' (google shopping)

  EXACT_KEYWORDS = df_list['Trends key'].to_list()
  Individual_EXACT_KEYWORD = list(zip(*[iter(EXACT_KEYWORDS)]*1))
  Individual_EXACT_KEYWORD = [list(x) for x in Individual_EXACT_KEYWORD]
  dicti = {}
  i = 1
  for Country in COUNTRY:
      for keyword in Individual_EXACT_KEYWORD:
          pytrend.build_payload(kw_list=keyword, 
                                timeframe = DATE_INTERVAL, 
                                geo = Country, 
                                cat=CATEGORY,
                                gprop=SEARCH_TYPE) 
          dicti[i] = pytrend.interest_over_time()
          i+=1
  df_trends = pd.concat(dicti, axis=1)
  df_trends.columns = df_trends.columns.droplevel(0) #drop outside header
  df_trends = df_trends.drop('isPartial', axis = 1) #drop "isPartial"
  df_trends.reset_index(level=0,inplace=True) #reset_index
  cols = ['date']
  for i in range(len(df_list)):
      cols.append(df_list['Symbol'][i])
  df_trends.columns=cols #change column names
  print(df_trends.head())
  df_google_trends = df_google_trends.append(df_trends)
  print(df_google_trends.head())
  time.sleep(1)

df_google_trends

2016-01-01 2016-07-01
        date  ASIANPAINT.BO  AXISBANK.BO  ...  TECHM.BO  TITAN.BO  ULTRACEMCO.BO
0 2016-01-01              0           52  ...         0        22             47
1 2016-01-02              0           32  ...         0        29             42
2 2016-01-03              0           23  ...        20        38              0
3 2016-01-04              0          100  ...        17        65              0
4 2016-01-05             48           54  ...         0        41              0

[5 rows x 31 columns]
        date  ASIANPAINT.BO  AXISBANK.BO  ...  TECHM.BO  TITAN.BO  ULTRACEMCO.BO
0 2016-01-01              0           52  ...         0        22             47
1 2016-01-02              0           32  ...         0        29             42
2 2016-01-03              0           23  ...        20        38              0
3 2016-01-04              0          100  ...        17        65              0
4 2016-01-05             48           54  ...         0        4

KeyboardInterrupt: ignored

In [ ]:
df_google_trends.to_csv('Google_trends.csv')